In [1]:
%run Util_func.ipynb

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# # Bay Area
# REG = 'BayArea'
# raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\Processing_20211018\spatial_join'

# out_file = r'..\out3\%s_4_TNCTrips_NumPax_Unprocessed.xlsx' %REG

# o_geo_col = 'otaz'
# d_geo_col = 'dtaz'

In [3]:
# SANDAG
REG = 'SANDAG'
raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SANDAG\RSG_HTS_Oct2021_sandag'

out_file = r'..\out3\%s_4_TNCTrips_NumPax_Unprocessed.xlsx' %REG

o_geo_col = 'o_bg_geo_id'
d_geo_col = 'd_bg_geo_id'

In [4]:
# # SCAG
# REG = 'SCAG'
# raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SCAG\RSG_HTS_Oct2021_scag'

# out_file = r'..\out3\%s_4_TNCTrips_NumPax_Unprocessed.xlsx' %REG

# o_geo_col = 'o_bg_geo_id'
# d_geo_col = 'd_bg_geo_id'

In [5]:
## Process trip records
trip_df = pd.read_csv(join(raw_dir, 'ex_trip_wZones.csv'))

trip_df = trip_df.loc[trip_df['mode_type_imputed']==9, ]

if REG == 'SANDAG':
    trip_df.loc[(trip_df['hh_id']==195023345) & (trip_df['person_num']==2) & (trip_df['day_num']==4), 'daywt_alladult_wkday'] = 700
    trip_df.loc[(trip_df['hh_id']==195025856) & (trip_df['person_num']==1) & (trip_df['day_num']==2), 'daywt_alladult_wkday'] = 700
    trip_df.loc[(trip_df['hh_id']==195041291) & (trip_df['person_num']==1) & (trip_df['day_num']==7), 'daywt_alladult_wkday'] = 700
    trip_df.loc[(trip_df['hh_id']==195064094) & (trip_df['person_num']==2) & (trip_df['day_num']==1), 'daywt_alladult_wkday'] = 700


trip_df['tnc_type'] = 3 # prem/other
trip_df.loc[(trip_df['mode_uber']==1) | (trip_df['mode_lyft']==1), 'tnc_type'] = 1 # pooled
trip_df.loc[(trip_df['mode_uber']==2) | (trip_df['mode_lyft']==2), 'tnc_type'] = 2 # regular

trip_df['tnc_pooled'][trip_df['tnc_pooled']>5] = 1
trip_df['tnc_occ'] = trip_df['num_travelers']
trip_df.loc[(trip_df['tnc_type']==1) & (trip_df['tnc_pooled']>0), 'tnc_occ'] += trip_df.loc[(trip_df['tnc_type']==1) & (trip_df['tnc_pooled']>0), 'tnc_pooled']
trip_df['tnc_occ_cat'] = trip_df['tnc_occ']
trip_df.loc[trip_df['tnc_occ_cat']>5, 'tnc_occ_cat'] = 5

PURP_DICT = {
            -1:-1,  #missing -> missing
            1:0,    #home -> home
            2:1,    #work -> work
            3:4,    #work-related -> work
            4:2,    #school -> school
            5:3,    #escort -> escort
            6:5,    #shop -> shop
            7:6,    #meal -> meal
            8:7,    #socrec -> socrec
            9:4,    #errand/other -> pers.bus.
            10:10,  #change mode -> change mode
            11:11,  #night non-home -> other
            12:11,  #other\missing -> other
            14:4,  #trip needs to be merged -> other
            }
trip_df.loc[pd.isna(trip_df['d_purpose_category_imputed']), 'd_purpose_category_imputed'] = -1
trip_df['dpurp'] = trip_df['d_purpose_category_imputed'].map(PURP_DICT)

trip_df.loc[pd.isna(trip_df['o_purpose_category_imputed']), 'o_purpose_category_imputed'] = -1
trip_df['opurp'] = trip_df['o_purpose_category_imputed'].map(PURP_DICT)
trip_df['dpurp2'] = trip_df['dpurp']
trip_df.loc[trip_df['dpurp']==0, 'dpurp2'] = trip_df.loc[trip_df['dpurp']==0, 'opurp']

trip_df['trexpfac'] = trip_df['daywt_alladult_wkday']

trip_df['count'] = 1
trip_df = trip_df[(trip_df['trexpfac']>0) & (trip_df['mode_type_imputed']>0)]
trip_df = trip_df[(trip_df[o_geo_col]>0) & (trip_df[d_geo_col]>0)]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
col_dict = {
    'dpurp': {
        'desc': 'DPurp',
        'col': 'dpurp',
        'vals': [-1] + list(range(0,8)) + [10,11],
        'labels': ['93_Missing','1_Home','2_Work','3_School','4_Escort','5_PersBus','6_Shop','7_Meal','8_SocRec',
                   '91_ChgMode','92_Other']
    },
    'dpurp2': {
        'desc': 'Non-Home Purp',
        'col': 'dpurp2',
        'vals': [-1] + list(range(1,8)) + [10,11],
        'labels': ['93_Missing','1_Work','2_School','3_Escort','4_PersBus','5_Shop','6_Meal','7_SocRec',
                   '91_ChgMode','92_Other']
    },
    'tnc_occ_cat': {
        'desc': 'NumPax',
        'col': 'tnc_occ_cat',
        'vals': range(1,6),
        'labels': ['1','2','3','4','5+']
    },
    'tncmode': {
        'desc': 'TNCMode',
        'col': 'tnc_type',
        'vals': range(1,4),
        'labels': ['2_Pool','1_Reg','3_PremOth']
    }
}

In [7]:
fname = out_file
writer = pd.ExcelWriter(fname, engine='xlsxwriter')
workbook = writer.book
format1 = workbook.add_format({'num_format': '#,##0.0'})

In [8]:
from xlsxwriter.utility import xl_rowcol_to_cell

row = 0
sname = 'Weekday'
d1_dict = col_dict['tnc_occ_cat']

title = 'TNC Trips by ' + d1_dict['desc']
tab = prep_data_1d(trip_df, d1_dict['desc'],d1_dict['col'], 'trexpfac', d1_dict['vals'], d1_dict['labels'])
row = write_to_excel(tab, sname, title, row)

title = 'Column Shares by ' + d1_dict['desc']
row = write_to_excel(getSharesIdx(tab.copy()), sname, title, row)

tab2 = tab.copy()
tab2.iloc[-1,-1] = tab2.iloc[-1,0]

title = 'Column Shares 95% CI by ' + d1_dict['desc']
row = write_to_excel(getSharesIdxCI95(tab.copy()), sname, title, row)

tab_range = xl_rowcol_to_cell(row,1) + ':' + xl_rowcol_to_cell(row,tab.shape[1])
_ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [9]:
wt_cols = ['count', 'trexpfac']
wt_desc = ['(Unweighted)', '(Weighted)']

for key in ['dpurp2', 'tncmode']:
    d2_dict = col_dict[key]
    row = 0
    sname = d2_dict['desc']
    
    for wc, wd in zip(wt_cols, wt_desc):
        title = 'TNC Trips by ' + d2_dict['desc'] + ' and ' + d1_dict['desc'] + ' ' + wd
        tab, tab_fmt = prep_data_2d(trip_df,d1_dict['col'],d1_dict['vals'],d1_dict['labels'],
                                d2_dict['col'],d2_dict['vals'],d2_dict['labels'],wc)
        row = write_to_excel(tab.astype('float64'), sname, title, row)

        if wc == 'count':
            tab2 = tab.copy()
        else:
            tab2.iloc[:-1,:-1] = tab.iloc[:-1,:-1]

        title = 'Column Shares by ' + d2_dict['desc'] + ' and ' + d1_dict['desc'] + ' ' + wd
        row = write_to_excel(getSharesIdx(tab.copy()), sname, title, row)

        title = 'Column Shares 95% CI by ' + d2_dict['desc'] + ' and ' + d1_dict['desc'] + ' ' + wd
        row = write_to_excel(getSharesIdxCI95(tab2.copy()), sname, title, row)
    tab_range = xl_rowcol_to_cell(row,1) + ':' + xl_rowcol_to_cell(row,tab.shape[1])
    _ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [10]:
writer.save()